In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
mobile_dat = pd.read_csv("Google Mobility.csv")
# c_cases = pd.read_csv("County Cases.csv")
# c_deaths = pd.read_csv("County Deaths.csv")
c_cases = pd.read_csv("confirmed_diff.csv")
c_deaths = pd.read_csv("deaths_diff.csv")

# clean mobility data
mobile_dat = mobile_dat.rename(columns={"retail_and_recreation_percent_change_from_baseline": "retail_and_recreation",
           "grocery_and_pharmacy_percent_change_from_baseline": "grocery_and_pharmacy",
           "parks_percent_change_from_baseline": "parks",
           "transit_stations_percent_change_from_baseline": "transit_stations",
           "workplaces_percent_change_from_baseline": "workplaces",
           "residential_percent_change_from_baseline": "residential"})
mobile_dat.drop(columns=['country_region_code', 'country_region'], inplace=True)
mobile_dat.rename(columns={'sub_region_1': 'state', 'sub_region_2': 'county'}, inplace=True)
mobile_dat['date'] = pd.to_datetime(mobile_dat['date'], format="%m/%d/%Y")

# clean cases data
c_cases.rename(columns={'Admin2': 'county', 'Province_State': 'state', 'Date': 'date', 'Value': 'cases', 'diff': 'case_change'}, inplace=True)
c_cases.drop(columns=['Country_Region'], inplace=True)
c_cases['date'] = pd.to_datetime(c_cases['date'], format="%m/%d/%Y")

# clean deaths data
c_deaths.rename(columns={'Admin2': 'county', 'Province_State': 'state', 'Date': 'date', 'Value': 'deaths', 'diff': 'death_change'}, inplace=True)
c_deaths.drop(columns=['Country_Region', 'Lat', 'Long_', 'region'], inplace=True)
c_deaths['date'] = pd.to_datetime(c_deaths['date'], format="%m/%d/%Y")

# Merge mobility data with cases and deaths data
obs = c_cases.merge(c_deaths, how='inner', on=['county', 'state', 'date'])
obs = obs.merge(mobile_dat, how='inner', on=['county', 'state', 'date'])

# Ensure proper order
obs.sort_values(['state', 'county', 'date'], ascending=[True, True, True], inplace=True)

# Format into observations and features
obs.drop(columns=['Lat', 'Long_', 'state', 'county', 'date'], inplace=True)
# obs = obs.set_index('region')

In [ ]:
# Split up by county
all_counties = sorted(obs['region'].unique())
batches = []
for county in all_counties:
    county_obs = obs[obs['region'] == county].copy()
    county_obs.drop(columns=['region'], inplace=True)
    batches.append(county_obs)
#     print(county_obs)
#     x_batches.append(county_obs.drop(columns=['cases']).values)
#     y_batches.append(county_obs['cases'].values)


# Follow Machine Learning Mastery
# multivariate data preparation
from numpy import array
from numpy import hstack
 
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        # seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        seq_x, seq_y = sequences[i:end_ix, 1:], sequences[end_ix-1, 0]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


# Baldwin County stuff
baldwin = obs[obs['region'] == 'Baldwin - Alabama']
baldwin = baldwin[baldwin['cases'] >= 30]
baldwin.drop(columns=['region'], inplace=True)
n_steps = 5
X, y = split_sequences(baldwin.values, n_steps)
# for i in range(len(X)):
#     print(X[i], y[i])
n_features = X.shape[2]


# All County Attempt
import random
random.shuffle(batches)
print(len(batches))
batches = batches[1:150]

n_steps = 5
X = []
y = []
for county in batches:
    X_temp, y_temp = split_sequences(county.values, n_steps)
    for i in range(len(X_temp)):
        X.append(X_temp[i])
        y.append(y_temp[i])
X = array(X)
y = array(y)
n_features = X.shape[2]

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
history = model.fit(X, y, epochs=500, verbose=1)
model.evaluate(X,y)
print(history.history.keys())
# # demonstrate prediction
# x_input = array([[80, 85], [90, 95], [100, 105]])
# x_input = x_input.reshape((1, n_steps, n_features))
# yhat = model.predict(x_input, verbose=0)
# print(yhat)

plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])

295
Epoch 1/500
10204/10204 [==============================] - 1s 109us/step - loss: 77344448.4502
Epoch 2/500
10204/10204 [==============================] - 1s 85us/step - loss: 35549911.4229
Epoch 3/500
10204/10204 [==============================] - 1s 86us/step - loss: 34240001.3519
Epoch 4/500
10204/10204 [==============================] - 1s 83us/step - loss: 30472814.5114
Epoch 5/500
10204/10204 [==============================] - 1s 83us/step - loss: 31851121.5914
Epoch 6/500
10204/10204 [==============================] - 1s 84us/step - loss: 33615243.8677
Epoch 7/500
10204/10204 [==============================] - 1s 87us/step - loss: 32321696.6351
Epoch 8/500
10204/10204 [==============================] - 1s 89us/step - loss: 31345491.2298
Epoch 9/500
10204/10204 [==============================] - 1s 85us/step - loss: 73682748.5257
Epoch 10/500
10204/10204 [==============================] - 1s 88us/step - loss: 29718775.0740
Epoch 11/500
10204/10204 [============================